by all i mean those that remain after anomaly removal......

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
from clean_data import __jday_convert

from datetime import datetime
from sgp4.api import Satrec, SatrecArray, WGS72

In [2]:
def get_satrec_erv(bst, ecc, aop, inc, mea, mem, raa, mmdot=0, mmddot=0, norad=0, epoch=None):
    '''
    Get cartesian coordinates of a satellite based on TLE parameters

     Parameters
     ----------
     bst : float : B-star
     ecc : float : eccentricity (in degrees)
     aop : float : argument of perigee (in degrees)
     inc : float : inclination (in degrees)
     mea : float : mean anomaly (in degrees)
     mem : float : mean motion (in degrees per minute)
     raa : float : right ascension of ascending node (in degrees)
     mmdot : float : NOT USED - ballistic coefficient
     mmddot : float : NOT USED - mean motion 2nd derivative
     norad : int : NOT USED - NORAD ID
     epoch : Timestamp : moment in time to get position

     Returns
     -------
     list
         [e, rx, ry, rz, vx, vy, yz] error, position xyz, velocity xyz.  error = 0 is good
    '''
    try:
        r = datetime.strptime('12/31/1949 00:00:00', '%m/%d/%Y %H:%M:%S')
        epoch_days = (epoch-r)/np.timedelta64(1, 'D')
        s = Satrec()
        s.sgp4init(
             WGS72,           # gravity model
             'i',             # 'a' = old AFSPC mode, 'i' = improved mode
             norad,               # satnum: Satellite number
             epoch_days,       # epoch: days since 1949 December 31 00:00 UT
             bst,      # bstar: drag coefficient (/earth radii)
             mmdot,   # ndot (NOT USED): ballistic coefficient (revs/day)
             mmddot,             # nddot (NOT USED): mean motion 2nd derivative (revs/day^3)
             ecc,       # ecco: eccentricity
             aop*np.pi/180, # argpo: argument of perigee (radians)
             inc*np.pi/180, # inclo: inclination (radians)
             mea*np.pi/180, # mo: mean anomaly (radians)
             mem*np.pi/(4*180), # no_kozai: mean motion (radians/minute)
             raa*np.pi/180, # nodeo: right ascension of ascending node (radians)
        )
        e,r,v = s.sgp4(*__jday_convert(epoch))
        return pd.Series([e, *r, *v, s])
    except:
        # e is SGP4 propagation errors, i've also added error 999 for when something goes wrong
        return pd.Series([999, 0,0,0,0,0,0, s])

In [ ]:
%%time

# Generate actual data

input_files = [
    (1, "test"),
    (1, "secret_test"),
    (2, "train"),
]

for n,f in input_files:
    print(f"Preparing data for: {f}")
    df = pd.read_pickle(f"{os.environ['GP_HIST_PATH']}/../{n}_min/{f}.pkl").set_index("GP_ID")
    saterv = df.progress_apply(lambda x:get_satrec_erv(bst=x.BSTAR,
                                                           ecc=x.ECCENTRICITY,
                                                           aop=x.ARG_OF_PERICENTER,
                                                           inc=x.INCLINATION,
                                                           mea=x.MEAN_ANOMALY,
                                                           mem=x.MEAN_MOTION,
                                                           raa=x.RA_OF_ASC_NODE,
                                                           epoch=x.EPOCH,), axis=1)
    saterv.columns = ["SAT_E","SAT_RX","SAT_RY","SAT_RZ","SAT_VX","SAT_VY","SAT_VZ","SATREC_OBJ"]
    display(saterv.head())

    saterv[["SAT_E","SAT_RX","SAT_RY","SAT_RZ","SAT_VX","SAT_VY","SAT_VZ"]].to_pickle(f"{os.environ['GP_HIST_PATH']}/../3_min/{f}_sgp4rv.pkl")
    saterv[["SATREC_OBJ"]].to_pickle(f"{os.environ['GP_HIST_PATH']}/../3_min/{f}_satrec.pkl")